In [1]:
!pip install adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.5 MB 9.1 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 39.2 MB/s 
     |████████████████████████████████| 264 kB 7.9 MB/s 
     |████████████████████████████████| 119 kB 52.2 MB/s 
     |████████████████████████████████| 243 kB 46.8 MB/s 


In [2]:
cd /content/drive/MyDrive/CAMemBERT2

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import AutoModelWithHeads,AutoConfig,Trainer,TrainingArguments,EarlyStoppingCallback
import numpy as np
import pdb
from sklearn.metrics import cohen_kappa_score,f1_score,accuracy_score,fbeta_score
from scipy.stats import spearmanr
from trainer_utils import get_dataset
import torch
import pandas as pd

def normalise_scores(example):
    score = example['labels']/40
    return {'labels':score}

dataset_obj,dataset_dict = get_dataset(['aes'],max_length=512)
# prepare for model
dataset_dict = dataset_dict.remove_columns(['dataset','labels']).rename_column('scores','labels')
dataset_dict = dataset_dict.map(normalise_scores)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Fa

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

In [ ]:
lr,bs = 6e-5,16
config = AutoConfig.from_pretrained('distilroberta-base',num_labels=1)
model = AutoModelWithHeads.from_pretrained('distilroberta-base',config=config)
adaptors_name='aes'
model.add_adapter(adaptors_name)
model.add_classification_head(
            adaptors_name,
            num_labels=1,
            overwrite_ok=True
        )
model.train_adapter(adaptors_name)

def compute_metrics(p):
    logits, labels = p.predictions,p.label_ids
    preds,labels = np.rint(logits.flatten()*40),np.rint(labels*40)
    metrics_dic = {
              "rmse": np.sqrt(np.mean((preds-labels)**2)),
              "pearson": np.corrcoef(preds,labels)[0,1],
              "spearman" : spearmanr(preds, labels)[0],
              "kappa":cohen_kappa_score(np.rint(preds),np.rint(labels),weights='quadratic')
              }
    return metrics_dic

label_names = ['labels']
args = TrainingArguments(
        output_dir = '/content',
        save_total_limit = 1,
        evaluation_strategy = "epoch",
        learning_rate = lr,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        remove_unused_columns=False,
        num_train_epochs = 15,
        weight_decay = 0.01,
        save_strategy = 'epoch',
        load_best_model_at_end=True,
        metric_for_best_model='pearson',
        label_names=label_names
    )


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['dev'],
    compute_metrics=compute_metrics
    )

early_stopping = EarlyStoppingCallback(early_stopping_patience = 5)
trainer.add_callback(early_stopping)
trainer.train()
trainer.evaluate(dataset_dict['test'])
pd.DataFrame(trainer.state.log_history).to_csv(f'results/aesadapters_distilroberta_lr{lr}_bs{bs}_norm.csv')


Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

Epoch,Training Loss,Validation Loss,Rmse,Pearson,Spearman,Kappa
1,No log,0.017467,5.326076,0.378110,0.390798,0.238722
2,No log,0.018134,5.356884,0.560708,0.560782,0.402809
3,No log,0.019417,5.602892,0.618277,0.610955,0.439468
4,No log,0.016967,5.238610,0.655046,0.664755,0.497642
5,No log,0.017581,5.322510,0.670651,0.672813,0.511731
6,No log,0.017325,5.268728,0.688167,0.682781,0.518679
7,No log,0.021113,5.807024,0.689226,0.692946,0.486109
8,0.022900,0.014707,4.867875,0.690029,0.686634,0.572937


***** Running Evaluation *****
  Num examples = 79
  Batch size = 16
Saving model checkpoint to /content/checkpoint-67
Configuration saved in /content/checkpoint-67/aes/adapter_config.json
Module weights saved in /content/checkpoint-67/aes/pytorch_adapter.bin
Configuration saved in /content/checkpoint-67/aes/head_config.json
Module weights saved in /content/checkpoint-67/aes/pytorch_model_head.bin
Configuration saved in /content/checkpoint-67/aes/head_config.json
Module weights saved in /content/checkpoint-67/aes/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 79
  Batch size = 16
Saving model checkpoint to /content/checkpoint-134
Configuration saved in /content/checkpoint-134/aes/adapter_config.json
Module weights saved in /content/checkpoint-134/aes/pytorch_adapter.bin
Configuration saved in /content/checkpoint-134/aes/head_config.json
Module weights saved in /content/checkpoint-134/aes/pytorch_model_head.bin
Configuration saved in /content/checkpoint-134/aes/h

In [ ]:
model.save_adapter('adapters/aes/','aes')